# Example - Extract interesting parts from transcripts, emails etc.

Here's what we're doing here:
 - Setting up an instance of AnythingLLM, on docker (https://github.com/Mintplex-Labs/anything-llm)
 - Filling the LLM "hopper" with all the data from an account (transcripts, emails, etc)
 - Adding these to the AnythingLLM "hot" directory to be embedded
 - Asking AnythingLLM to embed that file and keep it in the database for later
 - Asking AnythingLLM/OpenAI for details that should be in these documents, in particular when issues of interest were mentioned and where.
 - Returning all the examples of these things being mentioned.

## Notes:  Ensure AnythingLLM is running on a docker instance: /usr/local/bin/docker-compose up -d --build

## Remember, we need to have a persistent terminal running the following command that watches for file uploads:
## sudo docker exec -it --workdir=/app/collector anything-llm python watch.py

# TODO: 
The core algorithm should expect something like :
A title to use to describe this thing (like "Cost Concerns" or "competitor - snowflake" or whatever)
A list of strings corresponding to the thing (like ['this might be too expensive,' 'can we talk about overall cost, ...'])
A switch for determining if this is meant to be an exact match to some of this stuff or if we're trying to match the overall concept/fuzzy-match
An optional time frame to look for a second group in, to be delivered along with this one.
And then basically to return an object where we can get all the examples of the concept happening....
So do the following:
 - set up a json object for input, make a bunch of examples
 - make a nice csv file from parsing all the input files.
 - make a get_fuzzy function that gets all mentions, regexes those back out of the summary file and returns all the examples
 - make a get_exact function that does the same with regex. In both, return an object with a count and a set of stuff (can we add time?)
 - make an answer_questions manifold function that goes through as many questions as you have and returns a json object (or gives up) for each question/key.

In [1]:
from __future__ import print_function

from transcript_helpers import TranscriptEmbedder
import json
import os
import re

import requests
import json

import shutil

# where the text files are
anythingllm_summarydir = '/home/sean/repos/playground/sean/transcript_extraction'
# where AnythingLLM is installed
anythingllm_rootdir = '/home/sean/repos/anything-llm'

# the new domain name to construct
tenant_name = 'test_tenant_3'
# files we'd like to include in this new domain
files_to_encode = ['./avoma_transcript_3.txt','./avoma_transcript_4.txt']




## Remember, we need to have a persistent terminal running the following command that watches for file uploads:
## sudo docker exec -it --workdir=/app/collector anything-llm python watch.py

In [2]:
embedded_documents = TranscriptEmbedder()

# To set this up the first time
embedded_documents.set_up_by_filelist(files_to_encode,tenant_name,anythingllm_summarydir,anythingllm_rootdir)

# Subsequent times we'd like to address this person/tenant/domain again:
#embedded_documents.set_up_by_domain_name(tenant_name,anythingllm_summarydir,anythingllm_rootdir)


copying:  /home/sean/repos/playground/sean/transcript_extraction/avoma_transcript_3.txt
to:  /home/sean/repos/anything-llm/collector/hotdir
/home/sean/repos/anything-llm/collector/hotdir/avoma_transcript_3.txt
File Successfully Embedded.
copying:  /home/sean/repos/playground/sean/transcript_extraction/avoma_transcript_4.txt
to:  /home/sean/repos/anything-llm/collector/hotdir
/home/sean/repos/anything-llm/collector/hotdir/avoma_transcript_4.txt
File Successfully Embedded.
POST request successful.
Response Content: {"workspace":{"id":4,"name":"test_tenant_3","slug":"test_tenant_3-58562327","vectorTag":null,"createdAt":"2023-06-24 07:28:54","openAiTemp":null,"lastUpdatedAt":"2023-06-24 07:28:54"},"message":null}
Status Code: 200
Headers: {'X-Powered-By': 'Express', 'Vary': 'Origin', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '199', 'ETag': 'W/"c7-/4jds3UBSYr+i48x+GX9A5s/07E"', 'Date': 'Sat, 24 Jun 2023 07:28:54 GMT', 'Connection': 'keep-alive', 'Keep-Alive': 'tim

'/home/sean/repos/playground/sean/transcript_extraction/test_tenant_3_sumfile.csv'

# Test Queries

In [3]:
embedded_documents.return_fuzzy_mentions('The cost of the service')


POST request successful.
Response Content: {"id":"65746e7f-dcaa-41dc-bae5-506572f77166","type":"textResponse","textResponse":"[\n{\n    \"speaker_name\": \"Ashley Saville\",\n    \"conversation_text\": \"This is good to good to know and good to help us think about you know, from a cost perspective and from a a time to value implementation perspective of of getting the report solved, the report challenges solved\"\n},\n{\n    \"speaker_name\": \"Mona Akmal\",\n    \"conversation_text\": \"Oh, no. The sales team doesn't let we talk about pricing at all because I would give everything for free, but we don't live in Correct.\"\n}]","sources":[{"description":"a custom file uploaded by the user.","id":"bc6c21eb-758c-478e-9183-b0c6e92d84d3","published":"2023-06-22 20:25:22","title":"avoma_transcript_4.txt","token_count_estimate":9353,"url":"file:///app/collector/hotdir/processed/avoma_transcript_4.txt","wordCount":35622},{"description":"a custom file uploaded by the user.","id":"5dfd982a-9605

'[\n{\n    "speaker_name": "Ashley Saville",\n    "conversation_text": "This is good to good to know and good to help us think about you know, from a cost perspective and from a a time to value implementation perspective of of getting the report solved, the report challenges solved"\n},\n{\n    "speaker_name": "Mona Akmal",\n    "conversation_text": "Oh, no. The sales team doesn\'t let we talk about pricing at all because I would give everything for free, but we don\'t live in Correct."\n}]'